<hr>

Copyright &copy; 2020 [IBM Developer Skills Network](https://cognitiveclass.ai/?utm_source=bducopyrightlink&utm_medium=dswb&utm_campaign=bdu). This notebook and its source code are released under the terms of the [MIT License](https://bigdatauniversity.com/mit-license/).


<a href="https://cognitiveclass.ai"><img src = "https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/Logos/organization_logo/organization_logo.png" width = 400> </a>

<h1 align=center><font size = 5>Peer Review Final Assignment</font></h1>

## Introduction


In this lab, you will build an image classifier using the VGG16 pre-trained model, and you will evaluate it and compare its performance to the model we built in the last module using the ResNet50 pre-trained model. Good luck!

## Table of Contents

<div class="alert alert-block alert-info" style="margin-top: 20px">

<font size = 3>    

1. <a href="#item41">Download Data
2. <a href="#item42">Part 1</a>
3. <a href="#item43">Part 2</a>  
4. <a href="#item44">Part 3</a>  

</font>
    
</div>

<a id="item41"></a>

## Download Data

Use the <code>wget</code> command to download the data for this assignment from here: https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0321EN/data/concrete_data_week4.zip

Use the following cells to download the data.

In [31]:
# from google.colab import drive
# drive.mount('/content/drive')
import zipfile
import os

# Path to the zip file in Google Drive
zip_file_path = "/content/drive/My Drive/Copy of concrete_data_week4.zip"
# Directory where you want to extract the contents
extract_dir = "/content/drive/My Drive/concrete_data_week4/"

# Create the directory if it doesn't exist
if not os.path.exists(extract_dir):
    os.makedirs(extract_dir)

# Unzip the file
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)

print(f"File unzipped to {extract_dir}")

File unzipped to /content/drive/My Drive/concrete_data_week4/


After you unzip the data, you fill find the data has already been divided into a train, validation, and test sets.

<a id="item42"></a>

## Part 1

In this part, you will design a classifier using the VGG16 pre-trained model. Just like the ResNet50 model, you can import the model <code>VGG16</code> from <code>keras.applications</code>.

You will essentially build your classifier as follows:
1. Import libraries, modules, and packages you will need. Make sure to import the *preprocess_input* function from <code>keras.applications.vgg16</code>.
2. Use a batch size of 100 images for both training and validation.
3. Construct an ImageDataGenerator for the training set and another one for the validation set. VGG16 was originally trained on 224 × 224 images, so make sure to address that when defining the ImageDataGenerator instances.
4. Create a sequential model using Keras. Add VGG16 model to it and dense layer.
5. Compile the mode using the adam optimizer and the categorical_crossentropy loss function.
6. Fit the model on the augmented data using the ImageDataGenerators.

Use the following cells to create your classifier.

In [32]:
from keras.preprocessing.image import ImageDataGenerator
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.applications import VGG16
from keras.applications.vgg16 import preprocess_input

In [33]:
num_classes = 2

image_resize = 224

batch_size_training = 100
batch_size_validation = 100

In [71]:
data_generator = ImageDataGenerator(
    preprocessing_function=preprocess_input,
)

train_generator = data_generator.flow_from_directory(
    '/content/drive/My Drive/concrete_data_week4/concrete_data_week4/train',
    target_size=(image_resize, image_resize),
    batch_size=batch_size_training,
    class_mode='categorical')

validation_generator = data_generator.flow_from_directory(
    '/content/drive/My Drive/concrete_data_week4/concrete_data_week4/valid',
    target_size=(image_resize, image_resize),
    batch_size=batch_size_validation,
    class_mode='categorical')

Found 30001 images belonging to 2 classes.
Found 9501 images belonging to 2 classes.


In [73]:
model = Sequential()

In [74]:
model.add(VGG16(
    include_top=False,
    pooling='avg',
    weights='imagenet',
    ))

model.add(Dense(num_classes, activation='softmax'))

In [75]:
model.layers[0].trainable = False

In [76]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 512)               14714688  
                                                                 
 dense_2 (Dense)             (None, 2)                 1026      
                                                                 
Total params: 14715714 (56.14 MB)
Trainable params: 1026 (4.01 KB)
Non-trainable params: 14714688 (56.13 MB)
_________________________________________________________________


In [77]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [78]:
steps_per_epoch_training = len(train_generator)
steps_per_epoch_validation = len(validation_generator)
num_epochs = 2
steps_per_epoch_training

301

In [79]:
fit_history = model.fit(
    train_generator,
    steps_per_epoch=steps_per_epoch_training,
    epochs=num_epochs,
    validation_data=validation_generator,
    validation_steps=steps_per_epoch_validation,
    verbose=1,
)

Epoch 1/2
301/301 [==============================] - 446s 1s/step - loss: 0.0949 - accuracy: 0.9678 - val_loss: 0.0250 - val_accuracy: 0.9948
Epoch 2/2
301/301 [==============================] - 256s 849ms/step - loss: 0.0185 - accuracy: 0.9961 - val_loss: 0.0152 - val_accuracy: 0.9965


<a id="item43"></a>

## Part 2

In this part, you will evaluate your deep learning models on a test data. For this part, you will need to do the following:

1. Load your saved model that was built using the ResNet50 model.
2. Construct an ImageDataGenerator for the test set. For this ImageDataGenerator instance, you only need to pass the directory of the test images, target size, and the **shuffle** parameter and set it to False.
3. Use the **evaluate_generator** method to evaluate your models on the test data, by passing the above ImageDataGenerator as an argument. You can learn more about **evaluate_generator** [here](https://keras.io/models/sequential/).
4. Print the performance of the classifier using the VGG16 pre-trained model.
5. Print the performance of the classifier using the ResNet pre-trained model.


Use the following cells to evaluate your models.

In [53]:
from tensorflow.keras.models import load_model
resnet50_model = load_model('/content/classifier_resnet_model.keras')

In [81]:
# Evaluate the model on the test data
test_generator = data_generator.flow_from_directory(
    '/content/drive/My Drive/concrete_data_week4/concrete_data_week4/test',
    target_size=(image_resize, image_resize),
    shuffle = False,
  )

loss, accuracy = model.evaluate_generator(test_generator)
print(f'VGG16 pre-trained model: Test loss - {loss}, Test accuracy -  {accuracy}')


Found 500 images belonging to 2 classes.


<ipython-input-81-8fd847d58b20>:8: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  loss, accuracy = model.evaluate_generator(test_generator)


VGG16 pre-trained model: Test loss - 0.01156420074403286, Test accuracy -  0.9980000257492065


In [82]:
loss, accuracy = resnet50_model.evaluate_generator(test_generator)
print(f'ResNet pre-trained model.: Test loss - {loss}, Test accuracy -  {accuracy}')


<ipython-input-82-c26bc93087df>:1: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  loss, accuracy = resnet50_model.evaluate_generator(test_generator)


ResNet pre-trained model.: Test loss - 0.006428195163607597, Test accuracy -  0.9959999918937683


<a id="item44"></a>

## Part 3

In this model, you will predict whether the images in the test data are images of cracked concrete or not. You will do the following:

1. Use the **predict_generator** method to predict the class of the images in the test data, by passing the test data ImageDataGenerator instance defined in the previous part as an argument. You can learn more about the **predict_generator** method [here](https://keras.io/models/sequential/).
2. Report the class predictions of the first five images in the test set. You should print something list this:

<center>
    <ul style="list-style-type:none">
        <li>Positive</li>  
        <li>Negative</li>
        <li>Positive</li>
        <li>Positive</li>
        <li>Negative</li>
    </ul>
</center>

Use the following cells to make your predictions.

In [83]:
import numpy as np
# Predict the class of the images in the test set
vgg_predictions = model.predict_generator(test_generator)
resnet_predictions = resnet50_model.predict_generator(test_generator)

<ipython-input-83-4019ab01c51f>:3: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  vgg_predictions = model.predict_generator(test_generator)
<ipython-input-83-4019ab01c51f>:4: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  resnet_predictions = resnet50_model.predict_generator(test_generator)


In [84]:
# Get the class indices
class_indices = test_generator.class_indices
index_to_class = {v: k for k, v in class_indices.items()}

In [85]:
# Convert predictions to class labels
vgg_predicted_classes = np.argmax(vgg_predictions, axis=1)
vgg_predicted_labels = [index_to_class[idx] for idx in vgg_predicted_classes]

# Report the class predictions of the first five images in the test set
print("VGG Model First five predictions:")
for label in vgg_predicted_labels[:5]:
  print(label)

VGG Model First five predictions:
negative
negative
negative
negative
negative


In [86]:
# Convert predictions to class labels
resnet_predicted_classes = np.argmax(resnet_predictions, axis=1)
resnet_predicted_labels = [index_to_class[idx] for idx in resnet_predicted_classes]

# Report the class predictions of the first five images in the test set
print("Resnet Model First five predictions:")
for label in resnet_predicted_labels[:5]:
  print(label)

Resnet Model First five predictions:
negative
negative
negative
negative
negative


### Thank you for completing this lab!

This notebook was created by Alex Aklson.

This notebook is part of a course on **Coursera** called *AI Capstone Project with Deep Learning*. If you accessed this notebook outside the course, you can take this course online by clicking [here](https://cocl.us/DL0321EN_Coursera_Week4_LAB1).

<hr>

Copyright &copy; 2020 [IBM Developer Skills Network](https://cognitiveclass.ai/?utm_source=bducopyrightlink&utm_medium=dswb&utm_campaign=bdu). This notebook and its source code are released under the terms of the [MIT License](https://bigdatauniversity.com/mit-license/).